In [72]:
import requests
import pandas as pd

url = "https://download.bls.gov/pub/time.series/pr/pr.data.0.Current"

headers = {
    "User-Agent": "Tejal Kanase (tejal.s.kanase14@gmail.com)  - for data access"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    with open("pr.data.0.Current", "wb") as f:
        f.write(response.content)
    print("File downloaded successfully.")
else:
    print(f"Failed to download. Status code: {response.status_code}")

File downloaded successfully.


In [79]:
bls_df = pd.read_csv('pr.data.0.Current', sep='\t')

In [80]:
print(bls_df.describe())

               year         value
count  37239.000000  37239.000000
mean    2009.648353     32.277663
std        8.752694     44.670886
min     1995.000000    -54.300000
25%     2002.000000      0.700000
50%     2010.000000      3.900000
75%     2017.000000     83.301500
max     2025.000000    384.600000


In [74]:
print(bls_df.head())

   series_id          year period         value footnote_codes
0  PRS30006011        1995    Q01           2.6            NaN
1  PRS30006011        1995    Q02           2.1            NaN
2  PRS30006011        1995    Q03           0.9            NaN
3  PRS30006011        1995    Q04           0.1            NaN
4  PRS30006011        1995    Q05           1.4            NaN


In [75]:
print(bls_df.columns)

Index(['series_id        ', 'year', 'period', '       value',
       'footnote_codes'],
      dtype='object')


In [76]:
bls_df.columns = bls_df.columns.str.replace(' ', '')

In [77]:
print(bls_df.columns)

Index(['series_id', 'year', 'period', 'value', 'footnote_codes'], dtype='object')


In [10]:
bls_df['series_id'] = bls_df['series_id'].str.strip()
bls_df['period'] = bls_df['period'].str.strip()

In [20]:
grouped_bls_df = (
    bls_df.groupby(['series_id', 'year'])['value']
      .sum()
      .reset_index()
)

In [21]:
print(grouped_bls_df.head())

     series_id  year  value
0  PRS30006011  1995    7.1
1  PRS30006011  1996   -0.5
2  PRS30006011  1997    4.4
3  PRS30006011  1998    4.2
4  PRS30006011  1999   -7.7


In [26]:
best_year_df = (
    grouped_bls_df.loc[
        grouped_bls_df.groupby('series_id')['value'].idxmax()
    ]
)

In [27]:
best_year_df.reset_index()

,index,series_id,year,value
0,27,PRS30006011,2022,20.500
1,58,PRS30006012,2022,17.100
2,65,PRS30006013,1998,705.895
3,108,PRS30006021,2010,17.700
4,139,PRS30006022,2010,12.400
...,...,...,...,...
277,8414,PRS88003192,2002,282.800
278,8467,PRS88003193,2024,860.838
279,8496,PRS88003201,2022,37.200
280,8527,PRS88003202,2022,28.700


In [31]:
best_year_df.reset_index(drop=True).to_excel('R01_best_year_by_series_report.xlsx')

In [78]:
import json
url = "https://honolulu-api.datausa.io/tesseract/data.jsonrecords?cube=acs_yg_total_population_1&drilldowns=Year%2CNation&locale=en&measures=Population"
response = requests.get(url)
with open('population_data.json', 'w') as f:
    json.dump(response.json(), f)

In [35]:
 with open('population_data.json', 'r') as file:
            population_data = json.load(file)

In [36]:
print(population_data)

{'annotations': {'dataset_link': 'http://www.census.gov/programs-surveys/acs/', 'source_description': 'The American Community Survey (ACS) is conducted by the US Census and sent to a portion of the population every year.', 'source_name': 'Census Bureau', 'table_id': 'B01003', 'subtopic': 'Demographics', 'topic': 'Diversity', 'dataset_name': 'ACS 1-year Estimate'}, 'page': {'limit': 0, 'offset': 0, 'total': 10}, 'columns': ['Nation ID', 'Nation', 'Year', 'Population'], 'data': [{'Nation ID': '01000US', 'Nation': 'United States', 'Year': 2013, 'Population': 316128839.0}, {'Nation ID': '01000US', 'Nation': 'United States', 'Year': 2014, 'Population': 318857056.0}, {'Nation ID': '01000US', 'Nation': 'United States', 'Year': 2015, 'Population': 321418821.0}, {'Nation ID': '01000US', 'Nation': 'United States', 'Year': 2016, 'Population': 323127515.0}, {'Nation ID': '01000US', 'Nation': 'United States', 'Year': 2017, 'Population': 325719178.0}, {'Nation ID': '01000US', 'Nation': 'United State

In [38]:
population_df = pd.DataFrame(population_data['data'])

In [39]:
population_df

,Nation ID,Nation,Year,Population
0,01000US,United States,2013,316128839.0
1,01000US,United States,2014,318857056.0
2,01000US,United States,2015,321418821.0
3,01000US,United States,2016,323127515.0
4,01000US,United States,2017,325719178.0
5,01000US,United States,2018,327167439.0
6,01000US,United States,2019,328239523.0
7,01000US,United States,2021,331893745.0
8,01000US,United States,2022,333287562.0
9,01000US,United States,2023,334914896.0


In [40]:
filtered_pop_df = population_df.loc[(population_df['Year']>=2013) & (population_df['Year']<=2018)]

In [45]:
mean_population = filtered_pop_df['Population'].mean()
print("Population Mean = ", mean_population)

Population Mean =  322069808.0


In [44]:
std_population = round(filtered_pop_df['Population'].std(),2)
print("Population Standard Deviation = ",std_population)

Population Standard Deviation =  4158441.04


In [46]:
bls_df_filtered = bls_df.loc[(bls_df['series_id'] == 'PRS30006032') & (bls_df['period']== 'Q01')]

In [55]:
df_joined = pd.merge(bls_df_filtered, population_df, left_on = 'year', right_on = 'Year', how = 'left')

In [56]:
df_joined

,series_id,year,period,value,footnote_codes,Nation ID,Nation,Year,Population
0,PRS30006032,1995,Q01,0.0,NaN,NaN,NaN,NaN,NaN
1,PRS30006032,1996,Q01,-4.2,NaN,NaN,NaN,NaN,NaN
2,PRS30006032,1997,Q01,2.8,NaN,NaN,NaN,NaN,NaN
3,PRS30006032,1998,Q01,0.9,NaN,NaN,NaN,NaN,NaN
4,PRS30006032,1999,Q01,-4.1,NaN,NaN,NaN,NaN,NaN
5,PRS30006032,2000,Q01,0.5,NaN,NaN,NaN,NaN,NaN
6,PRS30006032,2001,Q01,-6.3,NaN,NaN,NaN,NaN,NaN
7,PRS30006032,2002,Q01,-6.6,NaN,NaN,NaN,NaN,NaN
8,PRS30006032,2003,Q01,-5.7,NaN,NaN,NaN,NaN,NaN
9,PRS30006032,2004,Q01,2.0,NaN,NaN,NaN,NaN,NaN


In [58]:
df_joined['Population'] = df_joined['Population'].astype('Int64')

In [59]:
df_report = df_joined[['series_id','year','period','value','Population']]

In [60]:
df_report

,series_id,year,period,value,Population
0,PRS30006032,1995,Q01,0.0,<NA>
1,PRS30006032,1996,Q01,-4.2,<NA>
2,PRS30006032,1997,Q01,2.8,<NA>
3,PRS30006032,1998,Q01,0.9,<NA>
4,PRS30006032,1999,Q01,-4.1,<NA>
5,PRS30006032,2000,Q01,0.5,<NA>
6,PRS30006032,2001,Q01,-6.3,<NA>
7,PRS30006032,2002,Q01,-6.6,<NA>
8,PRS30006032,2003,Q01,-5.7,<NA>
9,PRS30006032,2004,Q01,2.0,<NA>


In [62]:
df_report.to_excel('R02_population_by_year_series_report.xlsx')